### Configuration imports

In [45]:
from PIL import Image 
from IPython.display import display 
import random
import json
import pandas as pd
import numpy as np
import itertools as it
import pickle as pkl
import pprint as pp

### Utility funcs

In [255]:
# 1. Background 2. Families 3.Base 4. Occupation 5. Flowers 6. Doors

# collect metadata from csv
def collectMetadata(df):
    bg = df[['Background Title', 'Gen1 (background)', 'Gen2 (background)']].dropna(how='all')
    # print(bg)
    base = df[['Base Title', 'Gen1 (background).1', 'Gen2 (background).1', 
            'Fraction (from both Gen1 & Gen2)', 'Location', 'Storyline (Neighborhood location)']].dropna(how='all')
    # print(body)
    occupation = df[['Occupation Title', 'Gen1 (clothes)', 'Gen2 (clothes)']].dropna(how='all')
    # print(occupation)
    family = df[['Families Title', 'Gen1 (Ears)', 'Gen1 (body)', 'Pure %', 'Mixed %']].iloc[0:18]
    # print(family)
    flower = df[['Flowers Title', 'Gen1 (Flowers)', 'Gen2 (Flowers)', 'Flowers Title_tech', 
                'Gen1 (Flowers_tech)', 'Gen2 (Flowers_tech)']].dropna(how='all')
    # print(flower)
    doors = df[['Doors Title', 'Gen1 (Accessories)', 'Gen2 (Accessories)', 'Storyline (The house)']].dropna(how='all')
    # print(doors)
    return bg, base, occupation, family, flower, doors


# iterate over the gen1 metadata and construct the mappings of attributes I need:
# 1. Background 2. Families 3.Base 4. Occupation 5. Flowers 6. Doors
def collect_attributes(gen1_metadata):
    filteredMeta = []
    for i in range(len(gen1_metadata)):
        curr_meta = {}
        # faction
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Faction'), None)
        if index:
            curr_meta['faction'] = gen1_metadata[i]['attributes'][index]['value']
        # backgrounds
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Location'), None)
        if index:
            curr_meta['background'] = gen1_metadata[i]['attributes'][index]['value']
        # occupation
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Occupation'), None)
        if index:
            curr_meta['occupation'] = gen1_metadata[i]['attributes'][index]['value']
        # ears
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Ears'), None)
        if index:
            curr_meta['ears'] = gen1_metadata[i]['attributes'][index]['value']
        # family
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Family'), None)
        if index:
            curr_meta['family'] = gen1_metadata[i]['attributes'][index]['value']
        # flowers
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Flower'), None)
        if index:
            curr_meta['flower'] = gen1_metadata[i]['attributes'][index]['value']
        # accessories
        index = next((index for (index, d) in enumerate(gen1_metadata[i]['attributes']) if d['trait_type'] == 'Accessories'), None)
        if index:
            curr_meta['accessories'] = gen1_metadata[i]['attributes'][index]['value']
        filteredMeta.append(curr_meta)
    return filteredMeta


# now generate image metadata from the mappings
def generate_image_metadata(filteredMeta):
    images = []
    for i in range(len(filteredMeta)):
        curr_image = {}
        curr_image['tokenId'] = i+1
        curr_image['faction'] = filteredMeta[i]['faction']
        curr_image['background'] = filteredMeta[i]['background']
        curr_image['occupation'] = filteredMeta[i]['occupation']
        curr_image['ears'] = filteredMeta[i]['ears']
        curr_image['family'] = filteredMeta[i]['family']
        curr_image['flower'] = filteredMeta[i]['flower']
        curr_image['accessories'] = filteredMeta[i]['accessories']
        images.append(curr_image)
    return images


# not sure if rarity is needed here
rares = []
def getRarity(single):
    '''
        compute the rarity according to our standards
    '''
    acc_list = [13,18,26]
    tech_flower = range(32,64)
    shrooms = range(0,4)
    rare = bg['Drop rate'][single['Background']] + \
        body['Drop rate.1'][single['Body']] + \
        clothes['Drop rate.2'][single['Clothes']] +\
        ears['Drop rate.3'][single['Ears']] +\
        faces['Drop rate.4'][single['Faces']] +\
        eyes['Drop rate.5'][single['Eyes']] +\
        flowers['Drop rate.6'][single['Flowers']] +\
        accessories['Drop rate.8'][single['Accessories']] \
           
#     print(rare)
    out = 'we fucked up:/'
    if rare >= 0.48:
#         plain and common
        if single['Flowers'] not in shrooms:
            if single['Background'] != 5 and single['Clothes'] != 9 and \
            single['Ears'] != 3 and single['Flowers'] not in tech_flower \
            and single['Accessories'] not in acc_list:
                out = 'Casual'
            else:
                out = 'Casual tech'
    if rare < 0.48:
#         uncommon and rare
        if single['Flowers'] not in shrooms:
            if single['Background'] != 5 and single['Clothes'] != 9 and \
                single['Ears'] != 3 and single['Flowers'] not in tech_flower \
            and single['Accessories'] not in acc_list:
                out = 'Fancy'
            else:
                out = 'Fancy tech'
#     uber
    if single['Flowers'] in shrooms:
        if single['Background'] != 5 and single['Clothes'] != 9 and \
            single['Ears'] != 3 and single['Flowers'] not in tech_flower \
            and single['Accessories'] not in acc_list:
            out = 'Fungi'
#     legendary
    if single['Flowers'] in shrooms:
#         print('Flowers in shrooms')
        if single['Background'] == 5 or single['Clothes'] == 9 or \
            single['Ears'] == 3 or single['Flowers'] in tech_flower \
            or single['Accessories'] in acc_list:
            print('legendary')
            out = 'Fungi tech'
            
    faction = "Natural"
    if single['Background'] == 5 or single['Clothes'] == 9 or \
        single['Ears'] == 3 or single['Flowers'] in tech_flower \
        or single['Accessories'] in acc_list:
        faction="Tech"
        
        
#     print(out)
    rares.append(rare)
    return rare, out, faction

# Add a token Id to each image
def addTokenId(all_images, start_idx):
    i = start_idx
    for item in all_images:
    #     print(item)
        item['tokenId'] = i
        i = i + 1
    return all_images

def read_metadata(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data


# A function to generate unique image mappings
def create_new_image_gen(filteredMeta, bg, base, occupation, family, flower, doors, gen_string):
    # 1. Background 2. Families 3.Base 4. Occupation 5. Flowers 6. Doors
    # Create a new image metadata dict
    new_images_meta = []
    for i in range(len(filteredMeta)):
        # single image dict
        new_image = {}

        new_image ["Background"] = filteredMeta[i]['background']


        # if family depends on ears:
        if family['Gen1 (Ears)'].str.contains(filteredMeta[i]['ears']).any():
            s = family['Gen1 (Ears)'].str.contains(filteredMeta[i]['ears']).fillna(False)
            new_image['Family'] = family['Families Title'][s].values[0]
        else:
            # if Liberal - 50% chance for None
            if filteredMeta[i]['ears'] == 'None':
                
                new_image ['Family'] = random.choices(['None', 'Liberal'], weights=[0.5, 0.5], k=1)[0]
            else:
                # get the body by family name
                b = family['Families Title'].str.contains(filteredMeta[0]['family'])
                # if ears and body match - 10% chance for pure blood
                if family['Gen1 (body)'][b].values[0] == filteredMeta[i]['ears']:
                    
                    new_image ['Family'] = random.choices([filteredMeta[i]['family'], 'Pure_blood'], weights=[0.9, 0.1], k=1)[0]
                else:
                # otherwise 5% for mixed blood
                    new_image ['Family'] = random.choices([filteredMeta[i]['family'], 'Mixed_blood'], weights=[0.95, 0.05], k=1)[0]
        # base
        new_image ["Base"] = base.loc[(base['Location'] == filteredMeta[i]['background']) & 
                                      (base['Fraction (from both Gen1 & Gen2)'].str.contains(filteredMeta[i]['faction']))]['Base Title'].values[0]
        # add the neighborhood location randomly from the list
        neighborhoods = base.loc[(base['Location'] == filteredMeta[0]['background']) & 
        (base['Fraction (from both Gen1 & Gen2)'].str.contains(filteredMeta[0]['faction']))]['Storyline (Neighborhood location)'].values[0].split(', ')
        new_image ["Neighborhood"] = random.choice(neighborhoods)

        # occupation
        new_image ["Occupation"] = filteredMeta[i]['occupation']
        # flowers
        # if tech flower
        if filteredMeta[i]['flower'].find('tech') != -1:
            new_image ["Flower"] = flower.loc[(flower[gen_string+' (Flowers_tech)'] == filteredMeta[i]['flower'])]['Flowers Title_tech'].values[0]
        else:
            new_image ["Flower"] = flower.loc[(flower[gen_string+' (Flowers)'] == filteredMeta[i]['flower'])]['Flowers Title'].values[0]

        # doors
        if filteredMeta[i]['accessories'] == 'None':
            new_image ['Doors'] = doors.loc[(doors[gen_string+' (Accessories)'].isna())]['Doors Title'].values[0]
            new_image ['The house'] = doors.loc[(doors[gen_string+' (Accessories)'].isna())]['Storyline (The house)'].values[0]
        else:
            new_image ["Doors"] = doors.loc[(doors[gen_string+' (Accessories)'] == filteredMeta[i]['accessories'])]['Doors Title'].values[0]
            new_image ["The house"] = doors.loc[(doors[gen_string+' (Accessories)'] == filteredMeta[i]['accessories'])]['Storyline (The house)'].values[0]

        # add to the list
        new_images_meta.append(new_image)

    return new_images_meta



# generate OS compatible metadata
# {'Background': 'Chillden',
#  'Family': 'Juzephinos_mark',
#  'Base': 'Chillden',
#  'Neighborhood': 'Datura Uni',
#  'Occupation': 'Undecided',
#  'Flower': 'Green_cap',
#  'Doors': 'Seed',
#  'The house': 'This house has a secret entrance to the S.E.E.D.',
#  'tokenId': 1}
def writeFrontendMetadataTitles(all_images):
    meta = []
    for single in all_images:

        tokenId = str(single['tokenId'])
        a = {}
    #     add name
        a['name'] = "Greenhouse #"+tokenId
    #     add description
        a['description'] = single['The house']
        a['image'] = "ipfs://NewUriToReplace/"+tokenId+".png"
    #     a['image'] = "ipfs://QmcSGM8HTb1o4bH1JcTsrYX3SvGzr7KKmm7gxLxcDcd5HZ/"+tokenId+".jpg"
        
        a['edition'] = 1
        a['attributes'] = [
    #         background
            {
                'trait_type' : 'Background',
                'value' : single['Background'],
            },
            # family
            {
                'trait_type' : 'Family',
                'value' : single['Family']
            },
            # base
            {
                'trait_type' : 'Base',
                'value' : single['Base']
            },
            # neighborhood
            {
                'trait_type' : 'Neighborhood',
                'value' : single['Neighborhood']
            },
    #         clothes
            {
                'trait_type' : 'Occupation',
                'value' : single['Occupation']
            },
            # flower
            {
                'trait_type' : 'Flower',
                'value' : single['Flower']
            },
    #         door
            {
                'trait_type' : 'Door',
                'value' : single['Doors']
            },
    #         the house
            {
                'trait_type' : 'The house',
                'value' : single['The house']
            }
        ]

        meta.append(a)
    #     write to separate json each metadata
        with open('./metadata/'+tokenId+".json", 'w+') as outfile:
            json.dump(a, outfile, indent=4)

    return meta, all_images
        

def generateAllImages(all_images):
    # 1. Background 2. Families 3.Base 4. Occupation 5. Flowers 6. Doors
    # Generate Images
    # for idx in range(0, len(meta)+1):
    # for idx in range(0, 20):
    for idx in range(0, len(all_images)):
        tokenId = str(all_images[idx]['tokenId'])
        
        # concat layer names
        bg_name = all_images[idx]['Background']
        family_name = all_images[idx]['Family']
        base_name = all_images[idx]['Base']
        occupation_name = all_images[idx]['Occupation']
        flower_name = all_images[idx]['Flower']
        doors_name = all_images[idx]['Doors']

        im1 = Image.open(f'./layers/background/{bg_name}.png').convert('RGBA')
        im2 = Image.open(f'./layers/families/{family_name}.png').convert('RGBA')
        im3 = Image.open(f'./layers/base/{base_name}.png').convert('RGBA')
        im4 = Image.open(f'./layers/occupation/{occupation_name}.png').convert('RGBA')
        im5 = Image.open(f'./layers/flowers/{flower_name}.png').convert('RGBA')
        im6 = Image.open(f'./layers/doors/{doors_name}.png').convert('RGBA')
        a = [im1, im2, im3, im4, im5, im6]
        
        #Create each composite
        com1 = Image.alpha_composite(a[0], a[1])
        com2 = Image.alpha_composite(com1, a[2])
        com3 = Image.alpha_composite(com2, a[3])
        com4 = Image.alpha_composite(com3, a[4])
        com5 = Image.alpha_composite(com4, a[5])
        
        rgb_im = com5.convert('RGB')
        file_name = tokenId + ".png"
        rgb_im.save("./images/" + file_name)



In [259]:
generateAllImages(all_images)


In [256]:
# load the csv
df = pd.read_csv('greenhouses.csv', header=3)

# collect metadata from csv
bg, base, occupation, family, flower, doors = collectMetadata(df)

gen1_metadata = read_metadata('all-traits-gen1.json')

# filter the gen1 metadata
filteredMeta = collect_attributes(gen1_metadata)[:4010]

# generate image metadata from mappings
gen1_string = "Gen1"
new_images_meta = create_new_image_gen(filteredMeta, bg, base, occupation, family, flower, doors, gen1_string)

# add tokenId
start_idx=1
all_images_pre_meta = addTokenId(new_images_meta, start_idx)

# all metadata of gen1 greenhouses
meta, all_images = writeFrontendMetadataTitles(all_images_pre_meta)

#####################################
# gen2 metadata

gen2_metadata = read_metadata('all-traits-gen2.json')

# filter the gen2 metadata
filteredMeta_gen2 = collect_attributes(gen2_metadata)

# generate gen2 image metadata from mappings
gen2_string = "Gen2"
new_images_meta_gen2 = create_new_image_gen(filteredMeta_gen2, bg, base, occupation, family, flower, doors, gen2_string)

# add tokenId
new_idx = 4011
all_images_pre_meta_gen2 = addTokenId(new_images_meta_gen2, new_idx)

# all metadata of gen2 greenhouses
meta2, all_images_gen2 = writeFrontendMetadataTitles(all_images_pre_meta_gen2)


# concat gen1 and gen2 metadatas
meta += meta2
all_images += all_images_gen2

# save the metadata to a single json file
with open('./all_meta_greenhouses.json', 'w+') as outfile:
    json.dump(meta, outfile, indent=4)

# save the image generation metadata to a single json file
with open('./all_traits_greenhouses.json', 'w+') as outfile:
    json.dump(all_images, outfile, indent=4)


# Generate all images
# generateAllImages(all_images)


In [21]:
accessories["Not visible with flowers"][9]

['Cotton',
 'Cotton_tech Devils_trumpet',
 'Devils_trumpet_tech',
 'Water_lily',
 'Water_lily_tech',
 'King_protea',
 'King_protea_tech',
 'Forsythia',
 'Forsythia_tech',
 'Pine',
 'Pine_tech',
 'Boophone',
 'Boophone_tech',
 'Crassula_alstonii',
 'Crassula_alstonii_tech',
 'Bunny_succulent',
 'Bunny_succulent_tech']

In [202]:
accessories
# flowers
# clothes

# print(accessories.keys())
# list(range(len(bg['Title'])))
# print(bg['Drop rate'])
# faces = df.iloc[0:25, 28:32]
# faces = faces.to_dict(orient='list')
# faces['Drop rate.4'] = [round(float(i), 4) for i in faces['Drop rate.4']]
# print(ears)

{'Title.8': ['Roots',
  'Johny_downo',
  'Angel',
  'Petals',
  'Ocean',
  'Nirvana',
  'Diamond',
  'Goat',
  'Antlers',
  'Devil',
  'Doc',
  'Stylish',
  'Emo',
  'Thunder',
  'Booboo',
  'Scar',
  'Freckles',
  'Symbol',
  'Rose',
  'None',
  'Pacman',
  'Hairy',
  'Bangs',
  'Arrow',
  'Tentacles',
  'Fire',
  'Rain',
  'Heaven',
  'Snow'],
 'Drop rate.8': [0.01,
  0.03,
  0.015,
  0.007,
  0.013,
  0.027,
  0.008,
  0.028,
  0.005,
  0.05,
  0.02,
  0.02,
  0.01,
  0.01,
  0.032,
  0.03,
  0.07,
  0.06,
  0.002,
  0.35,
  0.07,
  0.01,
  0.029,
  0.01,
  0.004,
  0.02,
  0.01,
  0.03,
  0.02],
 'Storyline (Character traits)': ['Curious',
  'Energetic',
  'Adventurous',
  'Intelligent',
  'Artistic',
  'Smart',
  'Honest',
  'Fighter',
  'Shy',
  'Wild',
  'Proud',
  'Open',
  'Bossy',
  'Loud',
  'Odd',
  'Neat (visible with Artifact tech)',
  'Silly',
  'Lazy',
  'Daydreamer',
  'None',
  'Mellow',
  'Funny',
  'Quiet',
  'Sceptical',
  'Gentle',
  'Melancholic',
  'Straight for

In [37]:
## Generate Traits

#TODO
# add exceptions manually for:
## Clothes: Thief,Burglar
# compute rarity
# add fraction
# add family crossover
# external url  still under question?
# health random per nft or per face?



print(count)
# all_images_check = []
# while len(all_images) < TOTAL_IMAGES:
#     new_trait_image = create_new_image()
#     new_tuple = tuple(new_trait_image.values())
# #     check if the generated image has at least 5 unique attributes:
#     if new_tuple not in it.combinations(all_images_check, 5):
#         all_images.append(new_trait_image)
#         all_images_check.append(new_tuple)

400


In [191]:
# all_images[8019]

# len(bg['Title'])
# list(range(len(bg['Title'])))
# index(ears['Title.3'])
# ears['Title.3'].index(ears['Title.3'][-1])
# face
# all_images
# a = []
# for i in range(0, len(faces['Title.4'])):
#     a.append(False)
# len(faces['Title.4'])
# faces['']

# flowers_tech['Unnamed: 17']
# flowers_tech
# flowers
# flowers
# if not faces['Title.4'][2] in accessories['not_visible_with_face']:
#     print(faces['Title.4'])
# accessories
# eyes
# len(all_images)
# flowers
# print(accessories['not_visible_with_face'])
# if 'Balloon' in accessories['not_visible_with_face'][0]:
#     print('True')
# all_images
# faces
# accessories['Not visible with face.1'][all_images[151]["Accessories"]]
# all_images[0]['Accessories']
# faces['Title.4'][all_images[90]['Faces']]
# list(range(len(faces['Title.4'])))
# all_images
# bg['Title'][all_images[0]['Background']]
# accessories['Not visible with face.1']
# accessories['Not visible with flowers'][new_image["Accessories"]]
# accessories['Not visible with flowers'][21]
# if isinstance(accessories['Not visible with eyes'][14], list):
# #     print('exceptions')
#     if eyes['Title.5'][19] in accessories['Not visible with eyes'][14]:
#         print('monocle')
# # accessories

# # print(eyes['Title.5'][10])
# # print(accessories['Not visible with eyes'][3])

# if isinstance(accessories['Not visible with eyes'][16], list):
#     print('ok')
#     if eyes['Title.5'][18] in accessories['Not visible with eyes'][16]:
#             print('eye cond')
# if isinstance(accessories['Not visible with face.1'][4], list):
#         print('instance of faces')
#         if faces['Title.4'][22] in accessories['Not visible with face.1'][4]:
#             print('hit facecondition')
# print(clothes['Title.2'])
# if isinstance(accessories['Not visible with ears'][5], list):
#     print('ears')
#     if ears['Title.3'][3] in accessories['Not visible with ears'][5]:

#         print('ear cond ')
# print(clothes['Title.2'][10])
# accessories['Title.8'][4]
# print(faces['Title.4'][22])
# print(accessories['Not visible with face.1'][4])

# print(flowers['Title.6'][32])
# all_images[530]

In [204]:
# "1. Plain (if attribute drop rate count >= 0.3688 and there are no tech and mushroom attributes ) 
# 2. Common (if attribute drop rate count >=0.3688 and there are no mushroom attributes but there are
# tech attributes) 
# 3. Uncommon (if attribute drop rate count < 0.3688 and there are no tech and mushroom attributes) 
# 4. Rare (if attribute drop rate count < 0.3688 and there are no mushroom attributes but there are
# tech attributes)
# 5. Uber (if there is a mushroom attribute but no tech attributes)
# 6. Legendary (if there is a mushroom attribute and a tech attribute)
# 7. Artifact (if there is a Daturian color set with no tech attribute)
# 8. Heirloom (if there is a Daturian color set with a tech attribute - 
# there has to be only 1 Daturian per set for this type of rarity)"

# tech - flowers, 1 bg, 1 clothe, 2 accessories, 1 ear, 
# bg['Title'][5]
# clothes['Title.2'][9]
# ears['Title.3'][3]
# accessories['Title.8'][13]
# accessories['Title.8'][18]
# accessories['Title.8'][26]
# = [13,18,26]
# flowers['Title.6'][32:]

# mushrooms
# flowers['Title.6'][0:4]




In [38]:
shrooms = range(0,4)
print(shrooms)
if 0 in shrooms:
    print(1)
else:
    print('blah')

range(0, 4)
1


In [39]:
# with open("all_images.pickle", "rb") as input_file:
#     all_images = pkl.load(input_file)
all_images

[{'Background': 4,
  'Body': 3,
  'Clothes': 22,
  'Ears': 11,
  'Faces': 9,
  'Eyes': 2,
  'Flowers': 53,
  'Accessories': 19,
  'tokenId': 1},
 {'Background': 2,
  'Body': 5,
  'Clothes': 16,
  'Ears': 5,
  'Faces': 20,
  'Eyes': 7,
  'Flowers': 27,
  'Accessories': 17,
  'tokenId': 2},
 {'Background': 13,
  'Body': 7,
  'Clothes': 38,
  'Ears': 16,
  'Faces': 6,
  'Eyes': 22,
  'Flowers': 5,
  'Accessories': 22,
  'tokenId': 3},
 {'Background': 13,
  'Body': 9,
  'Clothes': 22,
  'Ears': 16,
  'Faces': 0,
  'Eyes': 3,
  'Flowers': 6,
  'Accessories': 27,
  'tokenId': 4},
 {'Background': 4,
  'Body': 5,
  'Clothes': 20,
  'Ears': 17,
  'Faces': 17,
  'Eyes': 12,
  'Flowers': 12,
  'Accessories': 15,
  'tokenId': 5},
 {'Background': 13,
  'Body': 6,
  'Clothes': 37,
  'Ears': 8,
  'Faces': 15,
  'Eyes': 12,
  'Flowers': 22,
  'Accessories': 19,
  'tokenId': 6},
 {'Background': 4,
  'Body': 0,
  'Clothes': 37,
  'Ears': 9,
  'Faces': 22,
  'Eyes': 7,
  'Flowers': 23,
  'Accessories': 2

In [208]:
# arr = np.array(rares)
# print(len(rares))
# print('Mean ', np.mean(arr))
# print('Median', np.median(arr))



In [209]:
# collect metadata
# all_images[1:3]
# {
#   "name": "Your Collection #1",
#   "description": "Remember to replace this description",
#   "image": "ipfs://NewUriToReplace/1.png",
#   "dna": "5f34550a61642e6484671b7b40ca1b6489efcb91",
#   "edition": 1,
#   "date": 1644589850384,
#   "attributes": [
#     {
#       "trait_type": "Background",
#       "value": "Black"
#     },
#     {
#       "trait_type": "Eyeball",
#       "value": "White"
#     },
#     {
#       "trait_type": "Eye color",
#       "value": "Yellow"
#     },
#     {
#       "trait_type": "Iris",
#       "value": "Small"
#     },
#     {
#       "trait_type": "Shine",
#       "value": "Shapes"
#     },
#     {
#       "trait_type": "Bottom lid",
#       "value": "Middle"
#     },
#     {
#       "trait_type": "Top lid",
#       "value": "High"
#     }
#   ],
#   "compiler": "HashLips Art Engine"
# }


#TODO
# add progress bar for health
# add factions natural/tech

# remove background, body, ear titles



In [211]:
# accessories['Title.8'][all_images[0]['Accessories']]
print(len(extras))

100


In [102]:
#### Generate Tricky Images    
# for idx in range(1091, len(extras)+1):
# # for idx in range(0, len(meta)+1):
# # for idx in range(0, 50):
#     tokenId = str(all_images[extras[idx]]['tokenId'])
    
#     bg_name = 'bg_' + bg['Title'][all_images[extras[idx]]['Background']].lower()
#     body_name = 'body_' + body['Title.1'][all_images[extras[idx]]['Body']].lower()
#     clothes_name = 'clothes_' + clothes['Title.2'][all_images[extras[idx]]['Clothes']].lower()
#     faces_name = 'faces_' + faces['Title.4'][all_images[extras[idx]]['Faces']].lower()
#     eyes_name = 'eyes_' + eyes['Title.5'][all_images[extras[idx]]['Eyes']].lower()
#     ears_name = 'ears_' + ears['Title.3'][all_images[extras[idx]]['Ears']].lower()
#     flowers_name = 'flowers_' + flowers['Title.6'][all_images[extras[idx]]['Flowers']].lower()
#     accessories_name = 'accessories_' + accessories['Title.8'][all_images[extras[idx]]['Accessories']].lower()

#     im1 = Image.open(f'./trait_layers/bg_layer/{bg_name}.png').convert('RGBA')
#     im2 = Image.open(f'./trait_layers/body_layer/{body_name}.png').convert('RGBA')
#     im3 = Image.open(f'./trait_layers/clothes_layer/{clothes_name}.png').convert('RGBA')
#     im4 = Image.open(f'./trait_layers/faces_layer/{faces_name}.png').convert('RGBA')
#     im5 = Image.open(f'./trait_layers/eyes_layer/{eyes_name}.png').convert('RGBA')
#     im6 = Image.open(f'./trait_layers/ears_layer/{ears_name}.png').convert('RGBA')
#     im7 = Image.open(f'./trait_layers/flowers_layer/{flowers_name}.png').convert('RGBA')
#     im8 = Image.open(f'./trait_layers/accessories_layer/{accessories_name}.png').convert('RGBA')
#     a = [im1, im2, im3, im4, im5, im6, im7, im8]

            
# #     if face above ears:
#     above_ears = faces['Layer above ears'][all_images[extras[idx]]['Faces']]
#     if above_ears:
# #         34567->45367
#         a[3], a[4], a[5], a[6], a[7] = a[4], a[5], a[3], a[6], a[7]

#     layer_no = int(accessories['Layer Number'][all_images[extras[idx]]['Accessories']])
#     if layer_no != 8:
# #         if under ears: 756
#         if layer_no == 6:
#             a[5], a[6], a[7] = a[7], a[5], a[6]
# #         if under flower
#         if layer_no == 7:
#             a[6], a[7] = a[7], a[6]
# #         if under body
#         if layer_no == 2:
#             a[2], a[3], a[4], a[5], a[6], a[7] = a[7], a[2], a[3], a[4], a[5], a[6]
#         if layer_no == 5:
# #             
#             a[5], a[6], a[7] = a[7], a[5], a[6]
            
    
#     #Create each composite
#     com1 = Image.alpha_composite(a[0], a[1])
#     com2 = Image.alpha_composite(com1, a[2])
#     com3 = Image.alpha_composite(com2, a[3])
#     com4 = Image.alpha_composite(com3, a[4])
#     com5 = Image.alpha_composite(com4, a[5])
#     com6 = Image.alpha_composite(com5, a[6])
#     com7 = Image.alpha_composite(com6, a[7])
    
#     #Convert to RGB
#     rgb_im = com7.convert('RGB')
#     file_name = tokenId + ".png"
#     rgb_im.save("./images2/" + file_name)

In [212]:

    #Convert to RGBaccessories = df.iloc[0:29, 60:70]
# # accessories['not_visible_with_face'].iloc[0]
# accessories = strToArr(accessories, 'Not visible with eyes')
# accessories = strToArr(accessories, 'Not visible with flowers')
# accessories = strToArr(accessories, 'Not visible with ears')
# accessories = strToArr(accessories, 'Not visible with face.1')
# accessories = strToArr(accessories, 'Not visible with clothes')


# accessories = preprocess(accessories, 'Layer Number')
# accessories = accessories.to_dict(orient='list')
# accessories['Drop rate.8'] = [float(i) for i in accessories['Drop rate.8']]
# print(accessories)


faces_kawaii
False
faces_uhm
False
faces_tongue


FileNotFoundError: [Errno 2] No such file or directory: './trait_layers/flowers_layer/flowers_shaggy_mane.png'

In [121]:
# print(int(accessories['layer_number'][all_images[1]['Accessories']]))
# flowers['Title.6'][14] = ''
# klausimai del Mary's tears and devils...
# klausimai kaip yra jei ir layer 8, ir layer eyes/ears persidengia
# kiek ko dedam i metadata, visus droprates?
# ar vardai su asteroid families make sense?

# print(flowers)
# meta[1]['Background']['Title']


# merge flowers with flowers tech before generating images!!!
# recheck exceptions for accessories
# update drop rates
# finish metadata
# generate names


# to receive
# a list of not working traits
# lino cuts
# website: design
# test env for the gallery
# metamask connection
# sales page

# accessories
# accessories = strToArr(accessories, 'Not visible with eyes')
# meta[0]['attributes']
faces

{'Title.4': ['Grimace',
  'Pumpkin',
  'Vampire',
  'Tongue',
  'Kawaii',
  'Bunny',
  'Hohoho',
  'Laugh',
  'Balloon',
  'Bunny_moustache',
  'Serious',
  'Joker',
  'Serious_pipe',
  'Moustache',
  'Confused',
  'Rabbit',
  'Happy',
  'Uhm',
  'Teardrop',
  'Saw',
  'Bedrock',
  'Sad',
  'Chill',
  'Pinocchio',
  'None'],
 'Drop rate.4': [0.055,
  0.06,
  0.04,
  0.08,
  0.0777,
  0.05,
  0.0323,
  0.05,
  0.006,
  0.02,
  0.042,
  0.02,
  0.004,
  0.01,
  0.02,
  0.09,
  0.045,
  0.075,
  0.05,
  0.02,
  0.03,
  0.045,
  0.07,
  0.008,
  0.0],
 'Storyline (stats: Health (random number from 10-100))': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'Layer above ears': [False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False

In [85]:
# generate all legacy guys
# 1.Background - 2.Body - 3.Clothes - 4.Face - 5.Eyes - 6.Ears - 7.Flowers - 8.Accessories
####### DATA LOADING ############
df_legacy = pd.read_csv('daturians_sets.csv', header=1)


# preprocess lists
def strToArr(arr, key):
    '''
    Simply convert to array of strings
    '''
    for index, row in arr.iterrows():
        if not pd.isna(row[key]):
    #         print(row['not_visible_with_face'])
            arr[key].iloc[index] = row[key].split(', ')
#             arr[key].iloc[index] = 8.0
#             print(arr[key].iloc[index])
            
    return arr

def preprocess(arr, key):
    '''
    Simply convert to array of strings
    '''
    for index, row in arr.iterrows():
        if pd.isna(row[key]):
    #         print(row['not_visible_with_face'])
#             arr[key].iloc[index] = row[key].split(', ')
            arr[key].iloc[index] = 8.0
            print(arr[key].iloc[index])
            
    return arr

In [86]:
df_legacy

,Unnamed: 0,Background,Unnamed: 2,Body,Unnamed: 4,Clothes,Unnamed: 6,Ears,Unnamed: 8,Faces,Unnamed: 10,Eyes,Unnamed: 12,Flower,Unnamed: 14,Accesories
0,3,Blue,5,Glass,33,Hoodie_Blue,6,Glass,4,Tongue,16,Joy,61,Frozen_holly_tech,26,Fire
1,5,Terracotta,9,Tattoo,38,None,11,Tattoo,10,Bunny moustache,18,Java,48,Tulip_tech,9,Antlers
2,6,Chip_tech,7,Tiger,22,Hoodie_Golden,8,Tiger,6,Bunny,10,Balloon,38,Bonsai_tech,10,Doc
3,7,Orange,7,Tiger,1,Mummy,8,Tiger,5,Kawaii,4,Sleepy,11,Cherry,3,Angel
4,7,Orange,7,Tiger,15,Tenue_de_ville,8,Tiger,5,Kawaii,4,Sleepy,43,Cherry_tech,3,Angel
5,8,Flamingo,10,Popsicle,7,Gigot_sleeve,12,Popsicle,9,Balloon,21,Glasses,12,Dahlia,12,Stylish
6,8,Flamingo,10,Popsicle,7,Gigot_sleeve,12,Popsicle,9,Balloon,8,Tired,44,Dahlia_tech,12,Stylish
7,9,Red,5,Glass,30,Hoodie_Red,14,Earrings_Red,3,Vampire,15,Pretty,10,Oak,10,Devil
8,9,Red,5,Glass,30,Hoodie_Red,14,Earrings_Red,18,Uhm,15,Pretty,42,Oak_tech,10,Devil
9,10,Cherry,9,Tattoo,14,Pinned,11,Tattoo,12,Joker,15,Pretty,9,Wine_cup,16,Scar


In [87]:
complete_legacy = [{} for x in range(18)]
# to replace:
# 531, 1210, 1924, 2225, 2250, 2770, 4053, 5726, 6049, 6425, 6862, 7175, 7417
# 3 265 295 522 784 791 1093 1273 1774 1885 1981 2040 2151 2196 3084 3181 3858 3966
complete_legacy[0]['tokenId'] = 3
complete_legacy[1]['tokenId'] = 265
complete_legacy[2]['tokenId'] = 295
complete_legacy[3]['tokenId'] = 522
complete_legacy[4]['tokenId'] = 784
complete_legacy[5]['tokenId'] = 791
complete_legacy[6]['tokenId'] = 1093
complete_legacy[7]['tokenId'] = 1273
complete_legacy[8]['tokenId'] = 1774
complete_legacy[9]['tokenId'] = 1885
complete_legacy[10]['tokenId'] = 1981
complete_legacy[11]['tokenId'] = 2040
complete_legacy[12]['tokenId'] = 2151
complete_legacy[13]['tokenId'] = 2196
complete_legacy[14]['tokenId'] = 3084
complete_legacy[15]['tokenId'] = 3181
complete_legacy[16]['tokenId'] = 3858
complete_legacy[17]['tokenId'] = 3966

# print(all_images[-1])
legacy = []
# # legacy['Background']
# # df_legacy
for idx, row in df_legacy.iterrows():
    print(idx)
    print(row)
    if not pd.isna(row[0]):
        bg1 = int(row[0]-1)
#         print(bg1)
    # else:
    #     bg1 = int(all_images[random.randint(0, len(all_images))]['Background'])
#         print(bg1)
    if not pd.isna(row[2]):
        bd1 = int(row[2]-1)
#         print(bd1)
    # else:
    #     bd1 = int(all_images[random.randint(0, len(all_images))]['Body'])
#         print(bd1)
    if not pd.isna(row[4]):
        cl1 = int(row[4]-1)
    # else:
    #     cl1 = all_images[random.randint(0, len(all_images))]['Clothes']
    if not pd.isna(row[6]):
        ea1 = int(row[6]-1)
    # else:
    #     ea1 = all_images[random.randint(0, len(all_images))]['Ears']
    if not pd.isna(row[8]):
        fac1 = int(row[8]-1)
    # else:
    #     fac1 = all_images[random.randint(0, len(all_images))]['Faces']
    if not pd.isna(row[10]):
        eye1 = int(row[10]-1)
    # else:
    #     eye1 = all_images[random.randint(0, len(all_images))]['Eyes']
    if not pd.isna(row[12]):
        flo = int(row[12]-1)
    # else:
    #     flo = all_images[random.randint(0, len(all_images))]['Flowers']
    if not pd.isna(row[14]):
        acc1 = int(row[14]-1)
#     else:
# #         acc1 = all_images[random.randint(0, len(all_images))]['Accessories']
#         acc1 = 19


    new_leg = {
        'Background': bg1,
        'Body': bd1,
        'Clothes': cl1,
        'Ears': ea1,
        'Faces': fac1,
        'Eyes': eye1,
        'Flowers': flo,
        'Accessories': acc1,
    }
    complete_legacy[idx].update(new_leg)
# print(legacy)
# tech_legacy = []
# for single in legacy:
#     a = single.copy()
#     a['Flowers'] = int(a['Flowers']+32)
#     tech_legacy.append(a)
    
# complete_legacy = legacy + tech_legacy 

0
Unnamed: 0                     3
Background                  Blue
Unnamed: 2                     5
Body                       Glass
Unnamed: 4                    33
Clothes              Hoodie_Blue
Unnamed: 6                     6
Ears                       Glass
Unnamed: 8                     4
Faces                     Tongue
Unnamed: 10                   16
Eyes                         Joy
Unnamed: 12                   61
Flower         Frozen_holly_tech
Unnamed: 14                   26
Accesories                  Fire
Name: 0, dtype: object
1
Unnamed: 0                   5
Background          Terracotta
Unnamed: 2                   9
Body                    Tattoo
Unnamed: 4                  38
Clothes                   None
Unnamed: 6                  11
Ears                    Tattoo
Unnamed: 8                  10
Faces          Bunny moustache
Unnamed: 10                 18
Eyes                      Java
Unnamed: 12                 48
Flower              Tulip_tech
Unnamed: 14

In [88]:
# complete_legacy = legacy
print(complete_legacy)


[{'tokenId': 3, 'Background': 2, 'Body': 4, 'Clothes': 32, 'Ears': 5, 'Faces': 3, 'Eyes': 15, 'Flowers': 60, 'Accessories': 25}, {'tokenId': 265, 'Background': 4, 'Body': 8, 'Clothes': 37, 'Ears': 10, 'Faces': 9, 'Eyes': 17, 'Flowers': 47, 'Accessories': 8}, {'tokenId': 295, 'Background': 5, 'Body': 6, 'Clothes': 21, 'Ears': 7, 'Faces': 5, 'Eyes': 9, 'Flowers': 37, 'Accessories': 9}, {'tokenId': 522, 'Background': 6, 'Body': 6, 'Clothes': 0, 'Ears': 7, 'Faces': 4, 'Eyes': 3, 'Flowers': 10, 'Accessories': 2}, {'tokenId': 784, 'Background': 6, 'Body': 6, 'Clothes': 14, 'Ears': 7, 'Faces': 4, 'Eyes': 3, 'Flowers': 42, 'Accessories': 2}, {'tokenId': 791, 'Background': 7, 'Body': 9, 'Clothes': 6, 'Ears': 11, 'Faces': 8, 'Eyes': 20, 'Flowers': 11, 'Accessories': 11}, {'tokenId': 1093, 'Background': 7, 'Body': 9, 'Clothes': 6, 'Ears': 11, 'Faces': 8, 'Eyes': 7, 'Flowers': 43, 'Accessories': 11}, {'tokenId': 1273, 'Background': 8, 'Body': 4, 'Clothes': 29, 'Ears': 13, 'Faces': 2, 'Eyes': 14, '

In [174]:
# generate metadata
for item in complete_legacy:
#     print(item)
    item['tokenId'] = random.randint(0, len(all_images))
#     i = i + 1

In [116]:
# for i in complete_legacy:
# complete_legacy
#     print(i['tokenId'])
faces

{'Title.4': ['Grimace',
  'Pumpkin',
  'Vampire',
  'Tongue',
  'Kawaii',
  'Bunny',
  'Hohoho',
  'Laugh',
  'Balloon',
  'Bunny_moustache',
  'Serious',
  'Joker',
  'Serious_pipe',
  'Moustache',
  'Confused',
  'Rabbit',
  'Happy',
  'Uhm',
  'Teardrop',
  'Saw',
  'Bedrock',
  'Sad',
  'Chill',
  'Pinocchio',
  'None'],
 'Drop rate.4': [0.055,
  0.06,
  0.04,
  0.08,
  0.0777,
  0.05,
  0.0323,
  0.05,
  0.006,
  0.02,
  0.042,
  0.02,
  0.004,
  0.01,
  0.02,
  0.09,
  0.045,
  0.075,
  0.05,
  0.02,
  0.03,
  0.045,
  0.07,
  0.008,
  0.0],
 'Storyline (stats: Health (random number from 10-100))': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'Layer above ears': [False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False

In [89]:
# generate images
#### Generate Images    
for idx in range(0, len(complete_legacy)):
    tokenId = str(complete_legacy[idx]['tokenId'])
    
    bg_name = 'bg_' + bg['Title'][complete_legacy[idx]['Background']].lower()
    body_name = 'body_' + body['Title.1'][complete_legacy[idx]['Body']].lower()
    clothes_name = 'clothes_' + clothes['Title.2'][complete_legacy[idx]['Clothes']].lower()
    faces_name = 'faces_' + faces['Title.4'][complete_legacy[idx]['Faces']].lower()
    eyes_name = 'eyes_' + eyes['Title.5'][complete_legacy[idx]['Eyes']].lower()
    ears_name = 'ears_' + ears['Title.3'][complete_legacy[idx]['Ears']].lower()
    flowers_name = 'flowers_' + flowers['Title.6'][complete_legacy[idx]['Flowers']].lower()
    accessories_name = 'accessories_' + accessories['Title.8'][complete_legacy[idx]['Accessories']].lower()

    im1 = Image.open(f'./trait_layers/bg_layer/{bg_name}.png').convert('RGBA')
    im2 = Image.open(f'./trait_layers/body_layer/{body_name}.png').convert('RGBA')
    im3 = Image.open(f'./trait_layers/clothes_layer/{clothes_name}.png').convert('RGBA')
    im4 = Image.open(f'./trait_layers/faces_layer/{faces_name}.png').convert('RGBA')
    im5 = Image.open(f'./trait_layers/eyes_layer/{eyes_name}.png').convert('RGBA')
    im6 = Image.open(f'./trait_layers/ears_layer/{ears_name}.png').convert('RGBA')
    im7 = Image.open(f'./trait_layers/flowers_layer/{flowers_name}.png').convert('RGBA')
    im8 = Image.open(f'./trait_layers/accessories_layer/{accessories_name}.png').convert('RGBA')
    a = [im1, im2, im3, im4, im5, im6, im7, im8]

            
#     if face above ears:
    above_ears = faces['Layer above ears'][complete_legacy[idx]['Faces']]
    f'above ears {above_ears}'
    if above_ears:
#         34567->45367
        a[3], a[4], a[5], a[6], a[7] = a[4], a[5], a[3], a[6], a[7]

    layer_no = int(accessories['Layer Number'][complete_legacy[idx]['Accessories']])
    if layer_no != 8:
#         if under ears: 756
        if layer_no == 6:
            a[5], a[6], a[7] = a[7], a[5], a[6]
#         if under flower
        if layer_no == 7:
            a[6], a[7] = a[7], a[6]
#         if under body
        if layer_no == 2:
            a[2], a[3], a[4], a[5], a[6], a[7] = a[7], a[2], a[3], a[4], a[5], a[6]
        if layer_no == 5:
#             
            a[5], a[6], a[7] = a[7], a[5], a[6]
            
    
    #Create each composite
    com1 = Image.alpha_composite(a[0], a[1])
    com2 = Image.alpha_composite(com1, a[2])
    com3 = Image.alpha_composite(com2, a[3])
    com4 = Image.alpha_composite(com3, a[4])
    com5 = Image.alpha_composite(com4, a[5])
    com6 = Image.alpha_composite(com5, a[6])
    com7 = Image.alpha_composite(com6, a[7])
    
    #Convert to RGB
    rgb_im = com7.convert('RGB')
    file_name = tokenId + ".png"
    rgb_im.save("./legacy_images/" + file_name)

In [90]:
# collect metadata
# all_images[1:3]
# {
#   "name": "Your Collection #1",
#   "description": "Remember to replace this description",
#   "image": "ipfs://NewUriToReplace/1.png",
#   "dna": "5f34550a61642e6484671b7b40ca1b6489efcb91",
#   "edition": 1,
#   "date": 1644589850384,
#   "attributes": [
#     {
#       "trait_type": "Background",
#       "value": "Black"
#     },
#     {
#       "trait_type": "Eyeball",
#       "value": "White"
#     },
#     {
#       "trait_type": "Eye color",
#       "value": "Yellow"
#     },
#     {
#       "trait_type": "Iris",
#       "value": "Small"
#     },
#     {
#       "trait_type": "Shine",
#       "value": "Shapes"
#     },
#     {
#       "trait_type": "Bottom lid",
#       "value": "Middle"
#     },
#     {
#       "trait_type": "Top lid",
#       "value": "High"
#     }
#   ],
#   "compiler": "HashLips Art Engine"
# }


#TODO
# add progress bar for health
# add factions natural/tech

# remove background, body, ear titles

meta_legacy = []
for single in complete_legacy:
#     rarity, faction = getRarity(single)
    if single['Flowers'] <32:
        faction = 'Natural'
        rarity = 'Artifact'
    else:
        faction = 'Tech'
        rarity = 'Artifact tech'
    tokenId = str(single['tokenId'])
#     combine ears and body
    if isinstance(ears['Family.1'][single["Ears"]], str):
        family1 = body['Family'][single['Body']]
        family2 = ears['Family.1'][single["Ears"]]
        if family1 == family2:
            family = family1
        else:
            family = family1 + ' x ' + family2
    else:
        family = body['Family'][single['Body']]
    
    a = {}
#     add name
    a['name'] = "Daturian #"+tokenId
#     add description
    a['description'] = body['Storyline (family)'][single['Body']]
    a['image'] = "ipfs://NewUriToReplace/"+tokenId+".png"
#     a['image'] = "ipfs://QmcSGM8HTb1o4bH1JcTsrYX3SvGzr7KKmm7gxLxcDcd5HZ/"+tokenId+".jpg"
    
    a['edition'] = 1
    a['attributes'] = [
#         rarity
        {
            'trait_type' : 'Type',
            'value' : rarity
        },
        {
            'trait_type' : 'Faction',
            'value' : faction
        },

#         background
        {
            'trait_type' : 'Location',
            'value' : bg['Location'][single['Background']]
        },
#         body
        {
            'trait_type' : 'Family',
            'value' : family
        },
#         clothes
        {
            'trait_type' : 'Occupation',
            'value' : clothes['Occupation'][single['Clothes']]
        },
#         ears
        {
            'trait_type' : 'Ears',
            'value' : ears['Title.3'][single['Ears']]
        },
#         face
        {
            'trait_type' : 'Face',
            'value' : faces['Title.4'][single['Faces']]
        },
        {
            'trait_type' : 'Health',
            'display_type': 'number',
            'value' : faces['Health'][single['Faces']]
        },

#         eyes
        {
            'trait_type' : 'Eyes',
            'value' : eyes['Title.5'][single['Eyes']]
        },
#         flowers
        {
            'trait_type' : 'Flower',
            'value' : flowers['Flower'][single['Flowers']]
        },
#         accessories
        {
            'trait_type' : 'Accessories',
            'value' : accessories['Title.8'][single['Accessories']]
        }
    ]
    a['extras'] = [
        {
            'trait_type' : 'Location description',
            'value' : bg['Storyline (location)'][single['Background']]
        },
        {
            'trait_type' : 'Family description',
            'value' : body['Storyline (family)'][single['Body']]
        },
        {
            'trait_type' : 'Occupation description',
            'value' : clothes['Storyline (occupation)'][single['Clothes']]
        },
        {
            'trait_type' : 'Special abilities',
            'value' : eyes['Storyline (special abilities)'][single['Eyes']]
        },
        {
            'trait_type' : 'Flora/fungi facts',
            'value' : flowers['Storyline (Flora/fungi facts)'][single['Flowers']]
        },
        {
            'trait_type' : 'Character traits',
            'value' : accessories['Storyline (Character traits)'][single['Accessories']]
        }
    ]

    meta.append(a)
#     write to separate json each metadata
    with open('./legacy_metadata/'+tokenId+".json", 'w+') as outfile:
        json.dump(a, outfile, indent=4)
    

In [91]:
# save complete_legacy for future
with open('./complete_legacy.json', 'w+') as outfile:
    json.dump(complete_legacy, outfile, indent=4)

In [87]:
# get rarity function
def getRarity(single):
#     compute the rarity according to our standards
    acc_list = [13,18,26]
    tech_flower = range(32,64)
    shrooms = range(0,4)
#     print(eyes['Drop rate.5'][single['Eyes']])
#     print(flowers['Flower'][single['Flowers']])
#     print(flowers['Drop rate.6'][single['Flowers']])
    
#     print(accessories['Drop rate.8'][single['Accessories']])
#     print(eyes['Drop rate.5'][single['Eyes']])
#     print(body['Drop rate.1'][single['Body']])
#     print(clothes['Drop rate.2'][single['Clothes']])
#     print(ears['Drop rate.3'][single['Ears']])
#     print(faces['Drop rate.4'][single['Faces']])
#     print(bg['Drop rate'][single['Background']])
#     if (flowers['Drop rate.6'][single['Flowers']] == 0):
#         print('drop rate=0')
#         print(flowers['Flower'][single['Flowers']])
    rare = 1.5/bg['Drop rate'][single['Background']] + \
        1.5/body['Drop rate.1'][single['Body']] + \
        1.5/clothes['Drop rate.2'][single['Clothes']] +\
        1.5/ears['Drop rate.3'][single['Ears']] +\
        1.5/faces['Drop rate.4'][single['Faces']] +\
        1.5/eyes['Drop rate.5'][single['Eyes']] +\
        1.5/flowers['Drop rate.6'][single['Flowers']] +\
        1.5/accessories['Drop rate.8'][single['Accessories']] \
           
#     print(rare)
    out = 'we fucked up:/'
#     if rare >= 0.48:
#         plain and common
    if single['Flowers'] not in shrooms:
        if single['Background'] != 5 and single['Clothes'] != 9 and \
        single['Ears'] != 3 and single['Flowers'] not in tech_flower \
        and single['Accessories'] not in acc_list:
            out = 'Casual'
        else:
            out = 'Casual tech'
#     if rare < 0.48:
#         uncommon and rare
    if single['Flowers'] not in shrooms:
        if single['Background'] != 5 and single['Clothes'] != 9 and \
            single['Ears'] != 3 and single['Flowers'] not in tech_flower \
        and single['Accessories'] not in acc_list:
            out = 'Fancy'
        else:
            out = 'Fancy tech'
#     uber
    if single['Flowers'] in shrooms:
        if single['Background'] != 5 and single['Clothes'] != 9 and \
            single['Ears'] != 3 and single['Flowers'] not in tech_flower \
            and single['Accessories'] not in acc_list:
            out = 'Fungi'
#     legendary
    if single['Flowers'] in shrooms:
#         print('Flowers in shrooms')
        if single['Background'] == 5 or single['Clothes'] == 9 or \
            single['Ears'] == 3 or single['Flowers'] in tech_flower \
            or single['Accessories'] in acc_list:
            print('legendary')
            out = 'Fungi tech'
            
    faction = "Natural"
    if single['Background'] == 5 or single['Clothes'] == 9 or \
        single['Ears'] == 3 or single['Flowers'] in tech_flower \
        or single['Accessories'] in acc_list:
        faction="Tech"
        
        
#     print(out)
    rares.append(rare)
    return rare, out, faction


# # update image metadata with ipfs url
# img_url = 'ipfs://Qmc6GR4znHrxpFKCWDYkn8eeLgGHahKBA7VT4PTc5xENcH/'
# # +tokenId+".png'
# for idx in range(0, len(all_images)):
# # for idx in range(0, 5):
#     current_id = all_images[idx]['tokenId']
#     curr_file = './metadata/'+str(current_id)+'.json'
#     with open(curr_file) as json_file:
#         tmp_data = json.load(json_file)
#         tmp_data['image'] = img_url+str(current_id)+'.png'
#         print(tmp_data['image'])
#     with open(curr_file, 'w') as out_file:
#         json.dump(tmp_data, out_file, indent=4)

In [85]:
# calculate the rank numbers:
# def myRarities(single_json):
#     rare = bg['Drop rate'][single['Background']] + \
#     body['Drop rate.1'][single['Body']] + \
#     clothes['Drop rate.2'][single['Clothes']] +\
#     ears['Drop rate.3'][single['Ears']] +\
#     faces['Drop rate.4'][single['Faces']] +\
#     eyes['Drop rate.5'][single['Eyes']] +\
#     flowers['Drop rate.6'][single['Flowers']] +\
#     accessories['Drop rate.8'][single['Accessories']] \
#     return rare

my_rarities = [];
all_images_file = './all_images.json'
with open(all_images_file) as json_file:
    all_images = json.load(json_file)
#     print(all_images['img_meta'][199])
    for idx, single in enumerate(all_images["img_meta"]):
#         print(idx)
        score, myType, faction = getRarity(single)
        my_rarities.append({'score': score, 'my_type': myType, 'faction': faction})
    print(my_rarities)
# for idx in range(0, len(all_images)):
# # for idx in range(0, 5):
#     current_id = all_images[idx]['tokenId']
#     curr_file = './metadata/'+str(current_id)+'.json'
#     with open(curr_file) as json_file:
#         tmp_data = json.load(json_file)
#         tmp_data['image'] = img_url+str(current_id)+'.png'
#         print(tmp_data['image'])
#     with open(curr_file, 'w') as out_file:
#         json.dump(tmp_data, out_file, indent=4)

legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
legendary
[{'score': 618.5781830462681, 'my_type': 'Fungi', 'faction': 'Natural'}, {'score': 151.52116402116403, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 207.27324263038548, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 173.89610389610388, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 187.03703703703704, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 242.99851615889352, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 182.69152595372108, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 204.7404009360531, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 149.05384423621507, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 217.57399577167018, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 149.28571428571428, 'my_type': 'F

In [62]:
# face none shouldn't have 0 drop rate
faces['Drop rate.4'][-1] = 0.0088

In [89]:
# now legacy
my_rarities_leg = [];
all_images_leg = './complete_legacy.json'
with open(all_images_leg) as json_file:
    all_images2 = json.load(json_file)
#     print(all_images['img_meta'][199])
    for idx, single in enumerate(all_images2):
#         print(idx)
        score, myType, faction = getRarity(single)
        my_rarities_leg.append({'score': score+300, 'my_type': myType, 'faction': faction})
    print(my_rarities_leg)

[{'score': 966.5598553098553, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 662.875939849624, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 867.2198172198173, 'my_type': 'Fungi', 'faction': 'Natural'}, {'score': 540.386334679813, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 1052.1634930567973, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 865.1515151515151, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 827.2767145135566, 'my_type': 'Fungi', 'faction': 'Natural'}, {'score': 802.4926186690893, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 1713.4852647352648, 'my_type': 'Fungi', 'faction': 'Natural'}, {'score': 731.3149350649351, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 610.952380952381, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 497.6379673785334, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 722.5840825840826, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 891.9670542635658, 'my_type': 'Fancy', 'faction': 'N

In [90]:
# append legacy scores:
for idx, single in enumerate(my_rarities_leg):
    my_rarities[all_images2[idx]['tokenId']-1] = single
print(my_rarities)

[{'score': 618.5781830462681, 'my_type': 'Fungi', 'faction': 'Natural'}, {'score': 151.52116402116403, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 207.27324263038548, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 173.89610389610388, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 187.03703703703704, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 242.99851615889352, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 182.69152595372108, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 204.7404009360531, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 149.05384423621507, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 217.57399577167018, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 149.28571428571428, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 191.22405372405373, 'my_type': 'Fancy tech', 'faction': 'Tech'}, {'score': 190.2668672790624, 'my_type': 'Fancy', 'faction': 'Natural'}, {'score': 253.1798360369789, 'my_type': 'Fa

In [104]:
# my_rarities[530]
# save rarities
with open('./my_rarities.json', 'w') as out_file:
    json.dump(my_rarities, out_file, indent=4)

In [100]:
print(my_rarities[1552])
print(my_rarities[36])

{'score': 225.32789282789284, 'my_type': 'Fancy tech', 'faction': 'Tech'}
{'score': 149.2668949272723, 'my_type': 'Fancy', 'faction': 'Natural'}


In [103]:
# append tokenId to each
for idx, single in enumerate(my_rarities):
    single['tokenId'] = idx +1
print(my_rarities)

[{'score': 618.5781830462681, 'my_type': 'Fungi', 'faction': 'Natural', 'tokenId': 1}, {'score': 151.52116402116403, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 2}, {'score': 207.27324263038548, 'my_type': 'Fancy tech', 'faction': 'Tech', 'tokenId': 3}, {'score': 173.89610389610388, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 4}, {'score': 187.03703703703704, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 5}, {'score': 242.99851615889352, 'my_type': 'Fancy tech', 'faction': 'Tech', 'tokenId': 6}, {'score': 182.69152595372108, 'my_type': 'Fancy tech', 'faction': 'Tech', 'tokenId': 7}, {'score': 204.7404009360531, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 8}, {'score': 149.05384423621507, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 9}, {'score': 217.57399577167018, 'my_type': 'Fancy tech', 'faction': 'Tech', 'tokenId': 10}, {'score': 149.28571428571428, 'my_type': 'Fancy', 'faction': 'Natural', 'tokenId': 11}, {'score': 191.22405372405373, 'my_